## Alternating Least Square

In [1]:
'''import libraries'''
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import SparseVector
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import broadcast
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.mllib.linalg import Matrices
spark = SparkSession.builder.appName('listenBrainz - exploratory data analysis').getOrCreate()
import pandas as pd
import numpy as np
import os
import sys
import pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, IntegerType

!pip install pyarrow
!pip install scikit-learn
!pip install implicit

from scipy.sparse import coo_matrix
import scipy.sparse as sparse
import implicit
from implicit.als import AlternatingLeastSquares

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix

from pyspark.ml.evaluation import RegressionEvaluator,RankingEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.mllib.evaluation import RankingMetrics

23/05/16 23:36:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/16 23:36:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
'''alternating least square'''
train_df = spark.read.parquet("/home/vr2229/final-project-group-29/final_project/final-project-group-29/train_df_train.parquet")
val_df = spark.read.parquet("/home/vr2229/final-project-group-29/final_project/final-project-group-29/test.parquet")

# train_df = spark.read.parquet('/home/vr2229/final-project-group-29/final_project/final-project-group-29/train_df_small.parquet')
# val_df = spark.read.parquet('/home/vr2229/final-project-group-29/final_project/final-project-group-29/val_df_small.parquet')
from pyspark.sql.functions import asc
training_df = train_df.sort(asc("user_id"))
validation_df = val_df.sort(asc("user_id"))

ranks = [20]
regParam = [0.1]
errors = []
min_error = float('inf')
training_df = training_df.repartition(20000)
print('repartition done')

for param in regParam:
    for rank in ranks:
        
        tempALS = ALS(maxIter=20, rank=rank, regParam=param,
              userCol='user_id', itemCol='codes', ratingCol='listens_mean', implicitPrefs = True, numUserBlocks = 10000,
              coldStartStrategy='drop', seed=1234)
        print('als done')
        training_df = training_df.coalesce(20000)

        model = tempALS.fit(training_df)
        print('fitting done')
        
        predictions = model.transform(validation_df)
        evaluator = RegressionEvaluator(metricName='rmse', labelCol='listens', 
                                predictionCol='prediction')
        print('evaluation done')
        rmse = evaluator.evaluate(predictions)
        print('rmse calculated')
        errors.append(rmse)
        print('errors appended')
        print ('For rank {0} and regParam {1} the RMSE is {2}'.format (rank, param, rmse))
        
        if rmse < min_error:
            min_error = rmse
            best_rank = rank
            best_regParam = param
            best_model = model
        
print ('The best model was trained with rank {0} and regParam {1}'.format(best_rank, best_regParam))

repartition done
als done


23/05/16 23:40:33 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/05/16 23:40:33 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/05/16 23:40:54 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/05/16 23:40:54 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


fitting done
evaluation done


rmse calculated
errors appended
For rank 20 and regParam 0.1 the RMSE is 10.03120082205401
The best model was trained with rank 20 and regParam 0.1
